In [ ]:
# Next steps: Switch while and for loops for new approach

In [9]:
import cv2
import os
import re
import time
import numpy as np

In [10]:
script_dir = os.path.dirname("Image Collection.ipynb")
data_dir = os.path.join(script_dir, "Data")

In [11]:
# Specify the list of gestures, a subfolder will be created for each one
gestures = ["1", "2", "3", "A", "B"]

# Specify the desired number of images for each gesture
desired_amount = {"1": 20, "2": 20, "3": 20, "A": 10, "B": 10}

# Initialize the dictionary of current number of occurrences per each gesture
current_amount = {gesture: 0 for gesture in gestures}

In [12]:
# Create Data folder if it does not exist yet
if not os.path.exists(data_dir):
    os.makedirs(data_dir)


for gesture in gestures:
    
    # Create a subfolder per each gesture if it does not exist yet
    new = os.path.join(data_dir, gesture)
    if not os.path.exists(new):
        os.makedirs(new)
        
    # If the subfolder exists, make sure that the ordering is correct and
    # shift it if any skips are present
    # (e.g. "A_1.jpg", "A_2.jpg", ... instead of "A_1.jpg", "A_3.jpg", ...)
    else:
        files = os.listdir(new)
        files.sort(key=lambda file: int(re.split(r"[_|.]", file)[1]))
        l = len(files)
        
        # Go through each file and if the run order skips a count, shift the respective file's run order
        for i in range(l - 1):
            name_split = re.split(r"[_|.]", files[i])
            name_split_next = re.split(r"[_|.]", files[i + 1])
            if (int(name_split[1]) + 1) != int(name_split_next[1]):
                new_name = name_split[0] + "_" + str(int(name_split[1]) + 1) + "." + name_split[2]
                os.rename(os.path.join(new, files[i + 1]), os.path.join(new, new_name))
                files = os.listdir(new)
                files.sort(key=lambda file: int(re.split(r"[_|.]", file)[1]))
            
        # Since the gesture subfolder is sorted by padding, we can use the last element as the current run
        current_amount[gesture] = 0 if not files else int(re.split(r"[_|.]", files[-1])[1])

paths = {gesture: os.path.join(data_dir, gesture) for gesture in gestures}

In [13]:
# The rectangle in the frame that is cropped from the web camera image
rect = [(225, 275), (425, 275), 
       (225, 475), (425, 475)]

In [14]:
cap = cv2.VideoCapture(0)

# Perform the data collecting process for each gesture in the given gesture list
for gesture in gestures:
    
    current = current_amount[gesture] + 1
    counter = current
    end = desired_amount[gesture]
    flag = 0
    exit = 0
    
    # Continue until the respective subfolder has the designated number of samples
    while counter <= end:
        ret, frame = cap.read()
        cv2.rectangle(frame, rect[0], rect[3], (0, 255, 0), 2)
        
        # Check validity
        if not ret:
            print("There has been a problem retrieving your frame")
            break
        
        # End the process for the current gesture in case the "q" key is hit
        key = cv2.waitKey(1)
        if key == ord("q"):
            break
        
        # End the whole process in case the "Esc" key is hit
        if key == ord("\x1b"):
            exit = 1
            break
            
        # Create grayscale version(s)
        frame_gray_gleam = (np.average((frame/255)**(1/2.2), axis=2)*255).astype(np.uint8)
        frame_gray_luminance = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # frame_gray_gleam - create correct approach for gamme correction using mask table and opencv LUT function
        
        # Binarize these version(s) using thresholding
        frame_binary_luminance = cv2.adaptiveThreshold(frame_gray_luminance, 1, 
                                                       cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 2)
        frame_binary_gleam = cv2.adaptiveThreshold(frame_gray_luminance, 1, 
                                                   cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 2)
        
        # Show all images
        # Live view with frame and text
        cv2.imshow("Real-time view", frame)
        #cv2.putText(frame, text=f"{gesture.capitalize()}", org=(rect[0][0], rect[0][1]), 
#                    font=cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, color=(0, 255, 0), thickness=2)
        
        # Grayscale version(s)
        cv2.imshow("Grayscale (luminance) view", frame_gray_luminance)
        cv2.imshow("Grayscale (gleam) view", frame_gray_gleam)
        
        # Binary version(s)
        cv2.imshow("Binary (luminance) view", frame_binary_luminance)
        cv2.imshow("Binary (gleam) view", frame_binary_gleam)
        
        if not flag:
            time.sleep(10)
        flag = 1
            
        # To reduce the number of almost identical frames, only save every n frames
        if not current % 4:
        
            # Create the naming for the file with the desired padding, i.e. ("gesture_run.jpg")
            img_name = gesture + "_" + str(counter) + ".jpg"
            img_path = r"%s" %os.path.join(paths[gesture], img_name)

            # Save the cropped rectangle from the frame
            if not cv2.imwrite(img_path, 
                               frame[(rect[0][1] + 2):(rect[2][1] - 2), 
                                     (rect[0][0] + 2):(rect[1][0] - 2)]):
                print("Something went wrong during this attempt:",
                      f"gesture - {gesture}, run - {counter}")
            
            counter += 1
            
        if exit:
            break
            
        current += 1

    # Close all the windows for the respective gesture
    if flag:
        cv2.destroyWindow(gesture.capitalize())
        cv2.destroyWindow(f"{gesture.capitalize()} grayscale (luminance)")
        cv2.destroyWindow(f"{gesture.capitalize()} grayscale (gleam)")
        cv2.destroyWindow(f"{gesture.capitalize()} binary (luminance)")
        cv2.destroyWindow(f"{gesture.capitalize()} binary (gleam)")
    
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Initialize some necessary variables
current = current_amount[gesture] + 1
counter = current
end = desired_amount[gesture]
flag = 0
exit = 0

cap = cv2.VideoCapture(0)

# Continue until the respective subfolder has the designated number of samples
while True:
    ret, frame = cap.read()
    cv2.rectangle(frame, rect[0], rect[3], (0, 255, 0), 2)

    # Check validity
    if not ret:
        print("There has been a problem retrieving your frame")
        break

    # End the process for the current gesture in case the "q" key is hit
    key = cv2.waitKey(1)
    if key == ord("q"):
        break

    # End the whole process in case the "Esc" key is hit
    if key == ord("\x1b"):
        exit = 1
        break
    
    # Create grayscale version(s)
    frame_gray_gleam = (np.average((frame/255)**(1/2.2), axis=2)*255).astype(np.uint8)
    frame_gray_luminance = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # frame_gray_gleam - create correct approach for gamme correction using mask table and opencv LUT function

    # Binarize these version(s) using thresholding
    frame_binary_luminance = cv2.adaptiveThreshold(frame_gray_luminance, 1, 
                                                   cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 2)
    frame_binary_gleam = cv2.adaptiveThreshold(frame_gray_luminance, 1, 
                                               cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 2)

    # Show all images
    # Live view with frame and text
    cv2.imshow("Real-time view", frame)
    #cv2.putText(frame, text=f"{gesture.capitalize()}", org=(rect[0][0], rect[0][1]), 
#                    font=cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, color=(0, 255, 0), thickness=2)

    # Grayscale version(s)
    cv2.imshow("Grayscale (luminance) view", frame_gray_luminance)
    cv2.imshow("Grayscale (gleam) view", frame_gray_gleam)

    # Binary version(s)
    cv2.imshow("Binary (luminance) view", frame_binary_luminance)
    cv2.imshow("Binary (gleam) view", frame_binary_gleam)
    
    for gesture in gestures:

# Perform the data collecting process for each gesture in the given gesture list
for gesture in gestures:
    
    current = current_amount[gesture] + 1
    counter = current
    end = desired_amount[gesture]
    flag = 0
    exit = 0
    
    # Continue until the respective subfolder has the designated number of samples
    while counter <= end:
        ret, frame = cap.read()
        cv2.rectangle(frame, rect[0], rect[3], (0, 255, 0), 2)
        
        # Check validity
        if not ret:
            print("There has been a problem retrieving your frame")
            break
        
        # End the process for the current gesture in case the "q" key is hit
        key = cv2.waitKey(1)
        if key == ord("q"):
            break
        
        # End the whole process in case the "Esc" key is hit
        if key == ord("\x1b"):
            exit = 1
            break
            
        # Create grayscale version(s)
        frame_gray_gleam = (np.average((frame/255)**(1/2.2), axis=2)*255).astype(np.uint8)
        frame_gray_luminance = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # frame_gray_gleam - create correct approach for gamme correction using mask table and opencv LUT function
        
        # Binarize these version(s) using thresholding
        frame_binary_luminance = cv2.adaptiveThreshold(frame_gray_luminance, 1, 
                                                       cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 2)
        frame_binary_gleam = cv2.adaptiveThreshold(frame_gray_luminance, 1, 
                                                   cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 2)
        
        # Show all images
        # Live view with frame and text
        cv2.imshow("Real-time view", frame)
        #cv2.putText(frame, text=f"{gesture.capitalize()}", org=(rect[0][0], rect[0][1]), 
#                    font=cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, color=(0, 255, 0), thickness=2)
        
        # Grayscale version(s)
        cv2.imshow("Grayscale (luminance) view", frame_gray_luminance)
        cv2.imshow("Grayscale (gleam) view", frame_gray_gleam)
        
        # Binary version(s)
        cv2.imshow("Binary (luminance) view", frame_binary_luminance)
        cv2.imshow("Binary (gleam) view", frame_binary_gleam)
        
        if not flag:
            time.sleep(10)
        flag = 1
            
        # To reduce the number of almost identical frames, only save every n frames
        if not current % 4:
        
            # Create the naming for the file with the desired padding, i.e. ("gesture_run.jpg")
            img_name = gesture + "_" + str(counter) + ".jpg"
            img_path = r"%s" %os.path.join(paths[gesture], img_name)

            # Save the cropped rectangle from the frame
            if not cv2.imwrite(img_path, 
                               frame[(rect[0][1] + 2):(rect[2][1] - 2), 
                                     (rect[0][0] + 2):(rect[1][0] - 2)]):
                print("Something went wrong during this attempt:",
                      f"gesture - {gesture}, run - {counter}")
            
            counter += 1
            
        if exit:
            break
            
        current += 1

    # Close all the windows for the respective gesture
    if flag:
        cv2.destroyWindow(gesture.capitalize())
        cv2.destroyWindow(f"{gesture.capitalize()} grayscale (luminance)")
        cv2.destroyWindow(f"{gesture.capitalize()} grayscale (gleam)")
        cv2.destroyWindow(f"{gesture.capitalize()} binary (luminance)")
        cv2.destroyWindow(f"{gesture.capitalize()} binary (gleam)")
    
cap.release()
cv2.destroyAllWindows()